<a href="https://colab.research.google.com/github/imgsude/gsk/blob/main/HastalikTahminiKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction")
model = AutoModelForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction", output_hidden_states=True) # add output_hidden_states=True to get hidden states
!pip install transformers torch scikit-learn
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score

from google.colab import files

uploaded = files.upload()
df = pd.read_csv("Duzenlenmis_Hastalik_Verisi.csv")
def extract_features(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.logits
    return outputs.hidden_states[-1][:, 0, :].numpy()
sentences = df.apply(lambda row: " ".join(row.astype(str)), axis=1).tolist()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#DIKKAT ** ARASI KODLAR OLMADAN MODEL %0 DOGRULUK ORANI VERIYOR CUNKU BALGAMLI OKSURUKLE OKSURUGU AYRI SAYIYOR. AYNI ANLAMA GELEN FARKLI KELIMELERI FARKLI SAYIYOR.
#*
# tfid BELIRTILER ICIN
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df["Belirti"])

# Cosine BENZER KELIMELERI BULUYOR
similarities = cosine_similarity(X_tfidf)

# BIRBIRINE YAKIN BELIRTILERI BULUYOR
for i in range(len(df)):
    most_similar_idx = similarities[i].argsort()[-2]  # KENDISINE ES OLMAYAN EN YAKIN
    print(f"{df['Belirti'][i]} → {df['Belirti'][most_similar_idx]}")
#*

print(sentences[:10])

X = np.array([extract_features(text) for text in sentences])
X = X.reshape(X.shape[0], -1)

labels = df['Belirti'].values
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy * 100:.2f}%")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Saving Duzenlenmis_Hastalik_Verisi.csv to Duzenlenmis_Hastalik_Verisi (3).csv
Ateş, Yorgunluk, Nefes Darlığı → Ateş, Yorgunluk, Nefes Darlığı
Öksürük, Yorgunluk → Öksürük, Yorgunluk
Öksürük, Yorgunluk → Öksürük, Yorgunluk
Ateş, Öksürük, Nefes Darlığı → Ateş, Öksürük, Nefes Darlığı
Ateş, Öksürük, Nefes Darlığı → Ateş, Öksürük, Nefes Darlığı
Ateş → Ateş
Ateş, Öksürük, Yorgunluk, Nefes Darlığı → Ateş, Öksürük, Yorgunluk, Nefes Darlığı
Ateş, Öksürük, Yorgunluk, Nefes Darlığı → Ateş, Öksürük, Yorgunluk, Nefes Darlığı
Öksürük → Öksürük
Öksürük → Öksürük
Ateş, Nefes Darlığı → Ateş, Nefes Darlığı
Öksürük, Yorgunluk → Öksürük, Yorgunluk
Öksürük → Öksürük
Belirti Yok → Belirti Yok
Belirti Yok → Belirti Yok
Öksürük → Öksürük
Ateş → Ateş
Öksürük, Yorgunluk, Nefes Darlığı → Öksürük, Yorgunluk, Nefes Darlığı
Ateş, Öksürük, Yorgunluk, Nefes Darlığı → Ateş, Öksürük, Yorgunluk, Nefes Darlığı
Ateş, Öksürük, Yorgunluk, Nefes Darlığı → Ateş, Öksürük, Yorgunluk, Nefes Darlığı
Ateş, Öksürük, Yorgunluk, Nefe